In [100]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

df = pd.read_excel('dataset.xlsx')
df.drop(["Patient ID"], axis=1, inplace=True)

In [101]:
print("O dataset possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))
df.isnull().sum()
quantidades_de_colunas_dropadas = 0
# Definindo a porcentagem máxima de valores nulos permitida
media_porcentagem_nulos_df = df.isna().mean().mean()

# Loop pelas colunas do DataFrame
for coluna in df.columns:
    # Calculando a porcentagem de valores nulos na coluna
    porcentagem_nulos = df[coluna].isnull().sum() / len(df)
    print(f"Coluna {coluna} não eliminada porcentagem de nulos: {porcentagem_nulos:.2%} de valores nulos.")
    # Verificando se a porcentagem de valores nulos na coluna é maior ou igual ao limite máximo permitido
    if porcentagem_nulos >= media_porcentagem_nulos_df:
        # Imprimindo mensagem informando que a coluna será eliminada
        print(f"Coluna {coluna} será eliminada por conter {porcentagem_nulos:.2%} de valores nulos.")
        # Removendo a coluna do DataFrame
        df.drop(columns=[coluna], inplace=True)
print("O dataset possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

O dataset possui 5644 linhas e 110 colunas
Coluna Patient age quantile não eliminada porcentagem de nulos: 0.00% de valores nulos.
Coluna SARS-Cov-2 exam result não eliminada porcentagem de nulos: 0.00% de valores nulos.
Coluna Patient addmited to regular ward (1=yes, 0=no) não eliminada porcentagem de nulos: 0.00% de valores nulos.
Coluna Patient addmited to semi-intensive unit (1=yes, 0=no) não eliminada porcentagem de nulos: 0.00% de valores nulos.
Coluna Patient addmited to intensive care unit (1=yes, 0=no) não eliminada porcentagem de nulos: 0.00% de valores nulos.
Coluna Hematocrit não eliminada porcentagem de nulos: 89.32% de valores nulos.
Coluna Hematocrit será eliminada por conter 89.32% de valores nulos.
Coluna Hemoglobin não eliminada porcentagem de nulos: 89.32% de valores nulos.
Coluna Hemoglobin será eliminada por conter 89.32% de valores nulos.
Coluna Platelets não eliminada porcentagem de nulos: 89.33% de valores nulos.
Coluna Platelets será eliminada por conter 89.33%

In [102]:
for col in df.select_dtypes(include="object").columns:
      df[col].replace({
            "positive": 1,
            "negative": 0,
            "detected": 1,
            "not_detected": 0,
            "present": 1,
            "absent": 0,
            "Não Realizado": float("NaN"),
            "not_done": float("NaN"),
            "<1000": 999
        }, inplace=True)


In [103]:
for coluna in df.columns:
    mediana = df[coluna].median
    df[coluna] = df[coluna].fillna(df[coluna].median())
print(df)

      Patient age quantile  SARS-Cov-2 exam result   
0                       13                       0  \
1                       17                       0   
2                        8                       0   
3                        5                       0   
4                       15                       0   
...                    ...                     ...   
5639                     3                       1   
5640                    17                       0   
5641                     4                       0   
5642                    10                       0   
5643                    19                       1   

      Patient addmited to regular ward (1=yes, 0=no)   
0                                                  0  \
1                                                  0   
2                                                  0   
3                                                  0   
4                                                  0   
...            

In [104]:
def find_outliers_IQR(column_name, df):
    q1, q3 = df[column_name].quantile([0.25, 0.75])
    IQR = q3 - q1
    outliers = df[(df[column_name] < q1 - 1.5*IQR) | (df[column_name] > q3 + 1.5*IQR)]
    return outliers

def print_outliers_IQR(df):
    for column in df.columns:
        outliers = find_outliers_IQR(column, df)
        print(f"Outliers da coluna {column}: {outliers.shape[0]}")

In [105]:
#print_outliers_IQR(df.describe())

In [106]:
# Naive Bayes
# separação das variáveis independentes e dependentes
X = df.iloc[:,:-1].values
y = df.iloc[:, -1].values

# separação dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# criação e treinamento do modelo Naive Bayes
model = GaussianNB()
model.fit(X_train, y_train)

# previsões com os dados de teste
y_pred = model.predict(X_test)

# avaliação da acurácia e matriz de confusão do modelo
#
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("Acurácia do modelo:", accuracy)
print("Matriz de confusão:\n", conf_matrix)
print("Revogação das Classes\n", class_report)

Acurácia do modelo: 0.23022432113341204
Matriz de confusão:
 [[ 375 1304]
 [   0   15]]
Revogação das Classes
               precision    recall  f1-score   support

         0.0       1.00      0.22      0.37      1679
         1.0       0.01      1.00      0.02        15

    accuracy                           0.23      1694
   macro avg       0.51      0.61      0.19      1694
weighted avg       0.99      0.23      0.36      1694



In [107]:
X = df.iloc[:,:-1].values
y = df.iloc[:, -1].values

# separação dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# criação e treinamento do modelo Naive Bayes
model = KNeighborsClassifier()
model.fit(X_train, y_train)

# previsões com os dados de teste
y_pred = model.predict(X_test)

# avaliação da acurácia e matriz de confusão do modelo
#
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("Acurácia do modelo:", accuracy)
print("Matriz de confusão:\n", conf_matrix)
print("Revogação das Classes\n", class_report)

Acurácia do modelo: 0.9911452184179457
Matriz de confusão:
 [[1679    0]
 [  15    0]]
Revogação das Classes
               precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      1679
         1.0       0.00      0.00      0.00        15

    accuracy                           0.99      1694
   macro avg       0.50      0.50      0.50      1694
weighted avg       0.98      0.99      0.99      1694



c:\Users\vhcla\Desktop\faculdade\Aprendizado de Máquina\ml-2023-1-trabalho-2\venv\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vhcla\Desktop\faculdade\Aprendizado de Máquina\ml-2023-1-trabalho-2\venv\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vhcla\Desktop\faculdade\Aprendizado de Máquina\ml-2023-1-trabalho-2\venv\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samp

In [108]:
X = df.iloc[:,:-1].values
y = df.iloc[:, -1].values

# separação dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# criação e treinamento do modelo Naive Bayes
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# previsões com os dados de teste
y_pred = model.predict(X_test)

# avaliação da acurácia e matriz de confusão do modelo
#
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("Acurácia do modelo:", accuracy)
print("Matriz de confusão:\n", conf_matrix)
print("Revogação das Classes\n", class_report)

NameError: name 'DecisionTreeClassifier' is not defined